In [1]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')

import torch
from utils.utils import run_eval,train,prepare
from models.Soft_TopK import TopK_custom

In [9]:
topk = TopK_custom(3,torch.device('cuda:0'),epsilon=1e-2,max_iter=100).to('cuda:0')

In [12]:
a = torch.tensor([[1,2,3,-float('inf'),5],[2,3,4,5,6]],dtype=torch.float,device='cuda:0')

In [13]:
topk(a)

tensor([[9.9977e-01, 9.4158e-01, 5.8639e-02, 1.0000e+00, 9.3097e-07],
        [1.0000e+00, 9.9977e-01, 9.4146e-01, 5.8518e-02, 2.4023e-04]],
       device='cuda:0')